In [1]:
# 检查jupyter notebook使用的python版本

import sys
print(sys.version)

3.6.9 (default, Jul 17 2020, 12:50:27) 
[GCC 8.4.0]


其实一开始在这上面还折腾了一会儿。

原因就是：
我在pyharm中新建了虚环境
然后之前的代码都是在这个ide里面运行的
但是后面在运行DQNDemo.py的时候显示出了问题
就是有一个库找不到
查了一下 应该是安装paddle的的时候出问题了
于是返回百度的这个项目里面，从简单的开始看起


所以，一步一步查看的需求出现
我就想到了jupyter
于是一开始的想法是
将pycharm中已经安装好的环境
直接导入anaconda
但是这个办法不行

所以现在这个办法就是
在pycharm中直接安装jupyter的包
然后在pycharm命令行里面
运行命令jupyter-notebook

于是就成功使用指定的虚环境了

In [4]:
pip list

Package             Version
------------------- ---------------
absl-py             0.9.0
appdirs             1.4.4
astor               0.8.1
attrs               19.3.0
Babel               2.8.0
backcall            0.2.0
bleach              3.1.5
certifi             2020.6.20
cfgv                3.1.0
chardet             3.0.4
click               7.1.2
cloudpickle         1.2.1
cycler              0.10.0
decorator           4.4.2
defusedxml          0.6.0
distlib             0.3.1
entrypoints         0.3
filelock            3.0.12
flake8              3.8.3
Flask               1.1.2
Flask-Babel         1.0.0
Flask-Cors          3.0.8
funcsigs            1.0.2
future              0.18.2
gast                0.3.3
graphviz            0.14.1
grpcio              1.30.0
gym                 0.17.2
identify            1.4.25
idna                2.10
importlib-metadata  1.7.0
importlib-resources 3.0.0
ipykernel           5.3.4
ipython             7.16.1
ipython-genutils    0.2.0
ipywidgets      

# 使用Paddle做基本子运算

In [2]:
import paddle.fluid as fluid

In [6]:
# 定义两个张量
x1 = fluid.layers.fill_constant(shape=[2, 2], value=1, dtype='int64')
x2 = fluid.layers.fill_constant(shape=[2, 2], value=1, dtype='int64')

感触：不要觉得这么简单的例子没有意义。
简单的例子，根本不是让你直接去解决这个问题
而是看懂别人的操作，知道以后怎么从简单到复杂来利用这个工具做事情
切忌眼高手低。

一看这个复杂就知道
其以后的功能就很强大
否则根本没有这么麻烦的必要

In [7]:
# 将两个张量求和
y1 = fluid.layers.sum(x=[x1, x2])

从上面来看，应该就是类似一个计算图之类的东西吧。
我先把未知数列出来
然后就是这些未知数之间的关系

然后下面就是开始告诉你
我准备用什么工具来烹饪你

In [8]:
# 创建一个使用CPU的解释器
place = fluid.CPUPlace()
exe = fluid.executor.Executor(place)
# 进行参数初始化
exe.run(fluid.default_startup_program())

/home/akira/PycharmProjects/OpenAIDemo/venv/lib/python3.6/site-packages/paddle/fluid/executor.py:1093: UserWarning: There are no operators in the program to be executed. If you pass Program manually, please use fluid.program_guard to ensure the current Program is being used.
  warnings.warn(error_info)


[]

In [9]:
# 进行运算，并把y的结果输出
result = exe.run(program=fluid.default_main_program(),
                 # 其实下面这个就是告诉输出什么
                 fetch_list=[y1])
print(result)

[array([[2, 2],
       [2, 2]], dtype=int64)]


In [3]:
import numpy as np

# 定义两个张量，并不指定该张量的形状和值，它们是之后动态赋值的。
# 这里只是指定它们的类型和名字，这个名字是我们之后赋值的关键。

a = fluid.layers.create_tensor(dtype='int64', name='a')
b = fluid.layers.create_tensor(dtype='int64', name='b')
y = fluid.layers.sum(x=[a, b])

place = fluid.CPUPlace()
exe = fluid.executor.Executor(place)
# 这里的意思是不是先预热一下，后面的run才是真正的执行
exe.run(fluid.default_startup_program())

/home/akira/PycharmProjects/OpenAIDemo/venv/lib/python3.6/site-packages/paddle/fluid/executor.py:1093: UserWarning: There are no operators in the program to be executed. If you pass Program manually, please use fluid.program_guard to ensure the current Program is being used.
  warnings.warn(error_info)


[]

In [13]:
a1 = np.array([3, 2]).astype('int64')
b1 = np.array([1, 1]).astype('int64')

out_a, out_b, result = exe.run(
    program=fluid.default_main_program(),
    # feed参数，这个就是要对张量变量进行赋值的。赋值的方式是使用了键值对的格式
    feed={'a': a1, 'b': b1},
    # fetch_list参数中，希望把a, b, y的值都输出来，所以要使用3个变量来接受返回值。
    fetch_list=[a, b, y]
)

print(out_a, " + ", out_b, " = ", result)

[3 2]  +  [1 1]  =  [4 3]


In [14]:
# x1张量究竟长什么样子
x1

name: "fill_constant_0.tmp_0"
type {
  type: LOD_TENSOR
  lod_tensor {
    tensor {
      data_type: INT64
      dims: 2
      dims: 2
    }
  }
}
persistable: false

In [16]:
# 查看x1张量所有的属性
dir(x1)

['__add__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__pow__',
 '__radd__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__rpow__',
 '__rsub__',
 '__rtruediv__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '_cloneVar',
 '_concatVar',
 '_detectContinuesSlice',
 '_detectEllipsis',
 '_get_info',
 '_reconstructSliceinfo',
 '_set_error_clip',
 '_set_info',
 '_sliceAndConcatVar',
 '_sliceVar',
 '_slice_indices',
 '_stop_gradient',
 'astype',
 'backward',
 'belong_to_optimizer',
 'block',
 'clear_gradient',
 'desc',
 'detach',
 'dtype',
 'error_clip',
 'grad_name',
 'gradient',
 'is_data',
 'lod_level',
 'name',
 'numpy

In [20]:
a1

array([3, 2])

In [21]:
type(y)

paddle.fluid.framework.Variable

In [22]:
type(a)

paddle.fluid.framework.Variable

In [23]:
type(x1)

paddle.fluid.framework.Variable

In [24]:
type(place)

paddle.fluid.core_avx.CPUPlace

In [25]:
type(exe)

paddle.fluid.executor.Executor

# 使用Paddle做线性回归

In [1]:
import paddle

In [11]:
# 定义一个简单的线性网络

# input layer size = 13 unites
x = fluid.layers.data(name='x', shape=[13], dtype='float32')
# hidden layer size = 100 unites
hidden = fluid.layers.fc(input=x, size=100, act='relu')
# output layer size = 1 unites
net = fluid.layers.fc(input=hidden, size=1, act=None)

# 定义损失函数
# label for the dataset
y = fluid.layers.data(name='y', shape=[1], dtype='float32')
# cost function using square_error
# input is the output of training network, label is y
cost = fluid.layers.square_error_cost(input=net, label=y)
avg_cost = fluid.layers.mean(cost)

In [12]:
# 复制一个主程序，方便之后使用
test_program = fluid.default_main_program().clone(for_test=True)

# 克隆一个程序，训练完成之后，作为预测程序进行预测程序

In [13]:
# 定义优化方法: 随机梯度下降优化方法
optimizer = fluid.optimizer.SGDOptimizer(learning_rate=0.01)
opts = optimizer.minimize(avg_cost)

In [14]:
# 创建新的CPU的解释器
place = fluid.CPUPlace()
exe = fluid.Executor(place)
# 进行参数初始化
exe.run(fluid.default_startup_program())

[]

In [15]:
# 定义训练和测试数据
x_data = np.array([[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 
                   [2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 
                   [3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 
                   [4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 
                   [5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]).astype('float32')

y_data = np.array([[3.0], [5.0], [7.0], [9.0], [11.0]]).astype('float32')

test_data = np.array([[6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]).astype('float32')

In [22]:
# 开始训练
for pass_id in range(10):
    train_cost = exe.run(program=fluid.default_main_program(),
                         feed={'x': x_data, 'y': y_data},
                         fetch_list=[avg_cost])
    print("Pass: %d, Cost: %0.5f" % (pass_id, train_cost[0]))

EnforceNotMet: 

--------------------------------------------
C++ Call Stacks (More useful to developers):
--------------------------------------------
0   std::string paddle::platform::GetTraceBackString<std::string const&>(std::string const&, char const*, int)
1   paddle::platform::EnforceNotMet::EnforceNotMet(std::string const&, char const*, int)
2   paddle::operators::SumOp::GetExpectedKernelType(paddle::framework::ExecutionContext const&) const
3   paddle::framework::OperatorWithKernel::ChooseKernel(paddle::framework::RuntimeContext const&, paddle::framework::Scope const&, paddle::platform::Place const&) const
4   paddle::framework::OperatorWithKernel::RunImpl(paddle::framework::Scope const&, paddle::platform::Place const&, paddle::framework::RuntimeContext*) const
5   paddle::framework::OperatorWithKernel::RunImpl(paddle::framework::Scope const&, paddle::platform::Place const&) const
6   paddle::framework::OperatorBase::Run(paddle::framework::Scope const&, paddle::platform::Place

上面的这个问题没什么有效的办法解决，怎么办，等到明天再看看吧。。。

In [19]:
type(avg_cost)

paddle.fluid.framework.Variable

In [20]:
[avg_cost]

[name: "mean_2.tmp_0"
 type {
   type: LOD_TENSOR
   lod_tensor {
     tensor {
       data_type: FP32
       dims: 1
     }
   }
 }
 persistable: false]